# Sistem Penilaian Resiko Kredit Berbasis Fuzzy Logic

## Deskripsi Tugas
Membangun sistem berbasis **Fuzzy Logic** untuk menilai resiko kredit dari 50 pengaju pinjaman berdasarkan:
- **Gaji (juta)**
- **Persentase cicilan pinjaman terhadap gaji (%)**

**Output:** File `peringkat.xlsx` berisi **10 orang dengan skor resiko kredit terkecil** yang akan diberikan pinjaman.

---

## Desain Sistem Fuzzy

### 1. Variabel Linguistik Input
| Variabel | Nama Linguistik | Keterangan |
|----------|-----------------|------------|
| Gaji (juta) | RENDAH, SEDANG, TINGGI | Semakin tinggi gaji, semakin aman |
| Cicilan (%) | RENDAH, SEDANG, TINGGI | Semakin tinggi cicilan, semakin beresiko |

### 2. Variabel Linguistik Output
| Variabel | Nama Linguistik | Rentang Skor |
|----------|-----------------|--------------|
| Resiko Kredit | AMAN, AGAK BERESIKO, BERESIKO | 0 - 100 |

### 3. Metode yang Digunakan
- **Fuzzification:** Mengubah nilai crisp menjadi derajat keanggotaan fuzzy
- **Inferensi:** Metode Mamdani dengan operator MIN-MAX
- **Defuzzification:** Metode Centroid (Center of Gravity)

---

In [40]:
import pandas as pd

### Membaca Data dari File Excel

In [42]:
def baca_data(nama_file):
    """
    Membaca data pengaju pinjaman dari file Excel.
    
    Parameter:
        nama_file (str): Nama file Excel yang akan dibaca
        
    Return:
        DataFrame: Data pengaju pinjaman
    """
    df = pd.read_excel(nama_file, header=1)
    
    # Standardisasi nama kolom (menghapus spasi ekstra dan menyesuaikan nama)
    df.columns = df.columns.str.strip()
    
    # Rename kolom
    df = df.rename(columns={
        'No Id pengaju pinjaman': 'No',
        'Gaji (juta)': 'Gaji (juta)',
        'Persentase cicilan pinjaman terhadap gaji(%)': 'Persentase cicilan pinjaman terhadap gaji(%)'
    })
    
    print(f"Kolom: {list(df.columns)}")
    return df

df = baca_data("resiko kredit.xlsx")
df.head()

Kolom: ['No', 'Gaji (juta)', 'Persentase cicilan pinjaman terhadap gaji(%)']


,No,Gaji (juta),Persentase cicilan pinjaman terhadap gaji(%)
0,1,3.7,54
1,2,6.2,48
2,3,2.9,58
3,4,9.5,44
4,5,4.1,52


In [43]:
# Informasi struktur data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   No                                            50 non-null     int64  
 1   Gaji (juta)                                   50 non-null     float64
 2   Persentase cicilan pinjaman terhadap gaji(%)  50 non-null     int64  
dtypes: float64(1), int64(2)
memory usage: 1.3 KB


In [44]:
# Statistik deskriptif data
df.describe()

,No,Gaji (juta),Persentase cicilan pinjaman terhadap gaji(%)
count,50.00000,50.00000,50.000000
mean,25.50000,10.10400,38.120000
std,14.57738,6.78278,15.733287
min,1.00000,1.20000,5.000000
25%,13.25000,4.50000,27.250000
50%,25.50000,8.35000,41.000000
75%,37.75000,14.60000,50.000000
max,50.00000,25.00000,62.000000


In [45]:
# Cek missing values
print(df.isnull().sum())

No                                              0
Gaji (juta)                                     0
Persentase cicilan pinjaman terhadap gaji(%)    0
dtype: int64


# BAGIAN 2: FUNGSI KEANGGOTAAN (MEMBERSHIP FUNCTION)

Fungsi keanggotaan digunakan untuk mengubah nilai crisp menjadi derajat keanggotaan fuzzy.

## Bentuk Kurva yang Digunakan:
1. **Kurva Trapesium** - untuk himpunan fuzzy di tepi (RENDAH dan TINGGI)
2. **Kurva Segitiga** - untuk himpunan fuzzy di tengah (SEDANG)

---

### Ilustrasi Kurva Trapesium
```
     1 |      ______
       |     /      \
       |    /        \
     0 |___/          \___
         a    b    c    d
```
- `a`: Titik awal (μ = 0)
- `b`: Titik awal puncak (μ = 1)
- `c`: Titik akhir puncak (μ = 1)  
- `d`: Titik akhir (μ = 0)

### Ilustrasi Kurva Segitiga
```
     1 |      /\
       |     /  \
       |    /    \
     0 |___/      \___
         a    b    c
```
- `a`: Titik awal (μ = 0)
- `b`: Titik puncak (μ = 1)
- `c`: Titik akhir (μ = 0)

In [46]:
# FUNGSI KEANGGOTAAN DASAR (KURVA TRAPESIUM DAN SEGITIGA)

def trapesium(x, a, b, c, d):
    """
    Fungsi keanggotaan kurva Trapesium.
    
    Rumus:
        - μ(x) = 0,                    jika x <= a atau x >= d
        - μ(x) = (x - a) / (b - a),    jika a < x <= b  (naik)
        - μ(x) = 1,                    jika b < x <= c  (puncak)
        - μ(x) = (d - x) / (d - c),    jika c < x < d   (turun)
    
    Parameter:
        x: Nilai input (crisp)
        a: Batas kiri bawah
        b: Batas kiri atas (awal puncak)
        c: Batas kanan atas (akhir puncak)
        d: Batas kanan bawah
        
    Return:
        float: Derajat keanggotaan (0 sampai 1)
    """
    if x <= a or x >= d:
        return 0.0
    elif a < x <= b:
        return (x - a) / (b - a)
    elif b < x <= c:
        return 1.0
    elif c < x < d:
        return (d - x) / (d - c)
    return 0.0


def segitiga(x, a, b, c):
    """
    Fungsi keanggotaan kurva Segitiga.
    
    Rumus:
        - μ(x) = 0,                    jika x <= a atau x >= c
        - μ(x) = (x - a) / (b - a),    jika a < x <= b  (naik)
        - μ(x) = (c - x) / (c - b),    jika b < x < c   (turun)
    
    Parameter:
        x: Nilai input (crisp)
        a: Batas kiri (μ = 0)
        b: Titik puncak (μ = 1)
        c: Batas kanan (μ = 0)
        
    Return:
        float: Derajat keanggotaan (0 sampai 1)
    """
    if x <= a or x >= c:
        return 0.0
    elif a < x <= b:
        return (x - a) / (b - a)
    elif b < x < c:
        return (c - x) / (c - b)
    return 0.0

# BAGIAN 3: PROSES FUZZIFIKASI

Fuzzifikasi adalah proses mengubah nilai **crisp (tegas)** menjadi **derajat keanggotaan fuzzy**.

## Desain Fungsi Keanggotaan Input

### Input 1: GAJI (dalam juta rupiah)
| Linguistik | Bentuk Kurva | Batas (a, b, c, d) |
|------------|--------------|-------------------|
| RENDAH | Trapesium | (0, 0, 4, 8) |
| SEDANG | Segitiga | (5, 10, 15) |
| TINGGI | Trapesium | (12, 18, 30, 30) |

```
μ     RENDAH        SEDANG          TINGGI
1 |____              /\               ____
  |    \            /  \             /
  |     \          /    \           /
0 |______\________/______\________/______
  0      4    8   5   10  12  15  18    30  → Gaji (juta)
```

### Input 2: CICILAN (dalam persentase terhadap gaji)
| Linguistik | Bentuk Kurva | Batas (a, b, c, d) |
|------------|--------------|-------------------|
| RENDAH | Trapesium | (0, 0, 20, 35) |
| SEDANG | Segitiga | (30, 45, 60) |
| TINGGI | Trapesium | (50, 65, 100, 100) |

```
μ     RENDAH        SEDANG          TINGGI
1 |____              /\               ____
  |    \            /  \             /
  |     \          /    \           /
0 |______\________/______\________/______
  0     20   35  30  45  60  50  65   100  → Cicilan (%)
```

In [52]:
# FUNGSI FUZZIFIKASI

def fuzzifikasi_gaji(gaji):
    """
    Mengubah nilai gaji (crisp) menjadi derajat keanggotaan fuzzy.
    
    Batas Fungsi Keanggotaan Gaji (dalam juta rupiah):
        - RENDAH : Trapesium (0, 0, 4, 8)   → Gaji <= 4 juta = rendah penuh
        - SEDANG : Segitiga  (5, 10, 15)    → Gaji sekitar 10 juta = sedang penuh
        - TINGGI : Trapesium (12, 18, 30, 30) → Gaji >= 18 juta = tinggi penuh
    
    Parameter:
        gaji (float): Nilai gaji dalam juta rupiah
        
    Return:
        dict: Derajat keanggotaan untuk setiap kategori
    """
    mu_rendah = trapesium(gaji, 0, 0, 4, 8)
    mu_sedang = segitiga(gaji, 5, 10, 15)
    mu_tinggi = trapesium(gaji, 12, 18, 30, 30)
    
    return {
        "rendah": mu_rendah, 
        "sedang": mu_sedang, 
        "tinggi": mu_tinggi
    }


def fuzzifikasi_cicilan(cicilan):
    """
    Mengubah nilai persentase cicilan (crisp) menjadi derajat keanggotaan fuzzy.
    
    Batas Fungsi Keanggotaan Cicilan (dalam persentase):
        - RENDAH : Trapesium (0, 0, 20, 35)     → Cicilan <= 20% = rendah penuh
        - SEDANG : Segitiga  (30, 45, 60)       → Cicilan sekitar 45% = sedang penuh
        - TINGGI : Trapesium (50, 65, 100, 100) → Cicilan >= 65% = tinggi penuh
    
    Parameter:
        cicilan (float): Nilai persentase cicilan terhadap gaji
        
    Return:
        dict: Derajat keanggotaan untuk setiap kategori
    """
    mu_rendah = trapesium(cicilan, 0, 0, 20, 35)
    mu_sedang = segitiga(cicilan, 30, 45, 60)
    mu_tinggi = trapesium(cicilan, 50, 65, 100, 100)
    
    return {
        "rendah": mu_rendah, 
        "sedang": mu_sedang, 
        "tinggi": mu_tinggi
    }

# Contoh penggunaan
print("Contoh :")
contoh_gaji = 7
contoh_cicilan = 40
print(f"Gaji = {contoh_gaji} juta → {fuzzifikasi_gaji(contoh_gaji)}")
print(f"Cicilan = {contoh_cicilan}% → {fuzzifikasi_cicilan(contoh_cicilan)}")

Contoh :
Gaji = 7 juta → {'rendah': 0.25, 'sedang': 0.4, 'tinggi': 0.0}
Cicilan = 40% → {'rendah': 0.0, 'sedang': 0.6666666666666666, 'tinggi': 0.0}


# BAGIAN 4: PROSES INFERENSI (ATURAN FUZZY)

Inferensi menggunakan **Metode Mamdani** dengan:
- **Operator AND:** MIN (mengambil nilai minimum)
- **Agregasi:** MAX (mengambil nilai maksimum)

## Tabel Aturan Inferensi (9 Aturan)

| No | IF Gaji | AND Cicilan | THEN Resiko |
|----|---------|-------------|-------------|
| R1 | RENDAH | RENDAH | AGAK BERESIKO |
| R2 | RENDAH | SEDANG | BERESIKO |
| R3 | RENDAH | TINGGI | BERESIKO |
| R4 | SEDANG | RENDAH | AMAN |
| R5 | SEDANG | SEDANG | AGAK BERESIKO |
| R6 | SEDANG | TINGGI | BERESIKO |
| R7 | TINGGI | RENDAH | AMAN |
| R8 | TINGGI | SEDANG | AMAN |
| R9 | TINGGI | TINGGI | AGAK BERESIKO |

### Logika Aturan:
- **Gaji tinggi + Cicilan rendah** = Paling AMAN (mampu bayar)
- **Gaji rendah + Cicilan tinggi** = Paling BERESIKO (sulit bayar)
- **Kondisi menengah** = AGAK BERESIKO

In [56]:
# FUNGSI INFERENSI (ATURAN FUZZY)

def inferensi(mu_gaji, mu_cicilan):
    """
    Menerapkan aturan fuzzy (Metode Mamdani).
    
    Menggunakan 9 aturan IF-THEN dengan:
        - Operator AND : MIN (mengambil nilai minimum dari antecedent)
        - Agregasi     : MAX (menggabungkan aturan dengan output yang sama)
    
    Parameter:
        mu_gaji (dict)    : Hasil fuzzifikasi gaji
        mu_cicilan (dict) : Hasil fuzzifikasi cicilan
        
    Return:
        dict: Derajat keanggotaan output (aman, agak_beresiko, beresiko)
    """
    
    # EVALUASI SETIAP ATURAN (menggunakan operator MIN)
    
    # Aturan 1: IF Gaji RENDAH AND Cicilan RENDAH THEN AGAK BERESIKO
    r1 = min(mu_gaji['rendah'], mu_cicilan['rendah'])
    
    # Aturan 2: IF Gaji RENDAH AND Cicilan SEDANG THEN BERESIKO
    r2 = min(mu_gaji['rendah'], mu_cicilan['sedang'])
    
    # Aturan 3: IF Gaji RENDAH AND Cicilan TINGGI THEN BERESIKO
    r3 = min(mu_gaji['rendah'], mu_cicilan['tinggi'])
    
    # Aturan 4: IF Gaji SEDANG AND Cicilan RENDAH THEN AMAN
    r4 = min(mu_gaji['sedang'], mu_cicilan['rendah'])
    
    # Aturan 5: IF Gaji SEDANG AND Cicilan SEDANG THEN AGAK BERESIKO
    r5 = min(mu_gaji['sedang'], mu_cicilan['sedang'])
    
    # Aturan 6: IF Gaji SEDANG AND Cicilan TINGGI THEN BERESIKO
    r6 = min(mu_gaji['sedang'], mu_cicilan['tinggi'])
    
    # Aturan 7: IF Gaji TINGGI AND Cicilan RENDAH THEN AMAN
    r7 = min(mu_gaji['tinggi'], mu_cicilan['rendah'])
    
    # Aturan 8: IF Gaji TINGGI AND Cicilan SEDANG THEN AMAN
    r8 = min(mu_gaji['tinggi'], mu_cicilan['sedang'])
    
    # Aturan 9: IF Gaji TINGGI AND Cicilan TINGGI THEN AGAK BERESIKO
    r9 = min(mu_gaji['tinggi'], mu_cicilan['tinggi'])
    
    # AGREGASI (menggabungkan aturan dengan operator MAX)
    
    # Output AMAN: R4, R7, R8
    output_aman = max(r4, r7, r8)
    
    # Output AGAK BERESIKO: R1, R5, R9
    output_agak_beresiko = max(r1, r5, r9)
    
    # Output BERESIKO: R2, R3, R6
    output_beresiko = max(r2, r3, r6)
    
    return {
        "aman": output_aman, 
        "agak_beresiko": output_agak_beresiko, 
        "beresiko": output_beresiko
    }


# Contoh penggunaan
print("CONTOH INFERENSI")
contoh_mu_gaji = fuzzifikasi_gaji(7)
contoh_mu_cicilan = fuzzifikasi_cicilan(40)
hasil_inferensi = inferensi(contoh_mu_gaji, contoh_mu_cicilan)
print(f"Gaji 7 juta, Cicilan 40% → {hasil_inferensi}")

CONTOH INFERENSI
Gaji 7 juta, Cicilan 40% → {'aman': 0.0, 'agak_beresiko': 0.4, 'beresiko': 0.25}


# BAGIAN 5: PROSES DEFUZZIFIKASI

Defuzzifikasi adalah proses mengubah hasil fuzzy (himpunan fuzzy) menjadi **nilai crisp (tegas)**.

## Metode yang Digunakan: CENTROID (Center of Gravity)

Rumus Centroid:
$$z^* = \frac{\sum_{i=1}^{n} z_i \cdot \mu(z_i)}{\sum_{i=1}^{n} \mu(z_i)}$$

Dimana:
- $z^*$ = Nilai output crisp (skor resiko)
- $z_i$ = Nilai domain output (0 sampai 100)
- $\mu(z_i)$ = Derajat keanggotaan pada titik $z_i$

## Fungsi Keanggotaan Output (Resiko Kredit: 0-100)

| Linguistik | Bentuk Kurva | Batas | Interpretasi |
|------------|--------------|-------|--------------|
| AMAN | Trapesium | (0, 0, 20, 40) | Skor 0-20 = Aman penuh |
| AGAK BERESIKO | Segitiga | (30, 50, 70) | Skor sekitar 50 |
| BERESIKO | Trapesium | (60, 80, 100, 100) | Skor 80-100 = Beresiko penuh |

```
μ       AMAN          AGAK BERESIKO       BERESIKO
1 |____                    /\                 ____
  |    \                  /  \               /
  |     \                /    \             /
0 |______\______________/______\___________/______
  0     20   40       30   50   70       60  80  100  → Skor Resiko
```

**Catatan:** Semakin RENDAH skor, semakin AMAN untuk diberikan pinjaman.

In [57]:
# FUNGSI KEANGGOTAAN OUTPUT (RESIKO KREDIT)

def mu_output_aman(x):
    """
    Fungsi keanggotaan output untuk kategori AMAN.
    Bentuk: Trapesium (0, 0, 20, 40)
    Skor rendah (0-20) = Aman penuh
    """
    return trapesium(x, 0, 0, 20, 40)


def mu_output_agak_beresiko(x):
    """
    Fungsi keanggotaan output untuk kategori AGAK BERESIKO.
    Bentuk: Segitiga (30, 50, 70)
    Skor sekitar 50 = Agak beresiko penuh
    """
    return segitiga(x, 30, 50, 70)


def mu_output_beresiko(x):
    """
    Fungsi keanggotaan output untuk kategori BERESIKO.
    Bentuk: Trapesium (60, 80, 100, 100)
    Skor tinggi (80-100) = Beresiko penuh
    """
    return trapesium(x, 60, 80, 100, 100)


# FUNGSI DEFUZZIFIKASI (METODE CENTROID)

def defuzzifikasi_centroid(hasil_inferensi):
    """
    Mengubah hasil inferensi fuzzy menjadi nilai crisp menggunakan metode Centroid.
    
    Langkah-langkah:
        1. Untuk setiap titik x (0 sampai 100):
           - Hitung derajat keanggotaan pada setiap fungsi output
           - Potong kurva dengan hasil inferensi (Alpha-cut / Clipping)
           - Gabungkan semua kurva dengan operator MAX (Union)
        2. Hitung Centroid: Sum(x * μ) / Sum(μ)
    
    Parameter:
        hasil_inferensi (dict): Hasil agregasi dari fungsi inferensi
        
    Return:
        float: Skor resiko kredit (0-100)
    """
    numerator = 0.0    # Pembilang: Σ(x * μ)
    denominator = 0.0  # Penyebut: Σ(μ)
    
    # Integral diskrit dengan sampling 0 sampai 100
    for x in range(0, 101):
        # Langkah 1: Hitung derajat keanggotaan pada setiap fungsi output
        mu_aman = mu_output_aman(x)
        mu_agak = mu_output_agak_beresiko(x)
        mu_beresiko = mu_output_beresiko(x)
        
        # Langkah 2: Alpha-cut (potong kurva dengan hasil inferensi)
        # Menggunakan operator MIN untuk clipping
        hasil_aman = min(mu_aman, hasil_inferensi['aman'])
        hasil_agak = min(mu_agak, hasil_inferensi['agak_beresiko'])
        hasil_beresiko = min(mu_beresiko, hasil_inferensi['beresiko'])
        
        # Langkah 3: Union (gabungkan semua kurva dengan MAX)
        mu_final = max(hasil_aman, hasil_agak, hasil_beresiko)
        
        # Langkah 4: Akumulasi untuk rumus Centroid
        numerator += x * mu_final
        denominator += mu_final
    
    # Hitung Centroid
    if denominator == 0:
        return 0
    
    skor = numerator / denominator
    return skor


# Contoh penggunaan
print("CONTOH DEFUZZIFIKASI :")
contoh_mu_gaji = fuzzifikasi_gaji(7)
contoh_mu_cicilan = fuzzifikasi_cicilan(40)
contoh_inferensi = inferensi(contoh_mu_gaji, contoh_mu_cicilan)
skor_resiko = defuzzifikasi_centroid(contoh_inferensi)
print(f"Gaji 7 juta, Cicilan 40% → Skor Resiko: {skor_resiko:.2f}")

CONTOH DEFUZZIFIKASI :
Gaji 7 juta, Cicilan 40% → Skor Resiko: 62.87


# BAGIAN 6: PROSES UTAMA DAN OUTPUT

Proses utama menggabungkan semua fungsi untuk:
1. Membaca data dari file Excel
2. Menghitung skor resiko untuk setiap pengaju pinjaman
3. Mengurutkan berdasarkan skor resiko (terkecil = paling aman)
4. Menyimpan 10 terbaik ke file `peringkat.xlsx`

In [59]:
# FUNGSI UTAMA: HITUNG SKOR RESIKO UNTUK SEMUA DATA

def hitung_skor_resiko(df):
    """
    Menghitung skor resiko kredit untuk seluruh data pengaju pinjaman.
    
    Proses:
        1. Untuk setiap baris data:
           - Ambil nilai gaji dan cicilan
           - Lakukan fuzzifikasi
           - Lakukan inferensi
           - Lakukan defuzzifikasi
        2. Simpan skor dan kategori ke dataframe
    
    Parameter:
        df (DataFrame): Data pengaju pinjaman
        
    Return:
        DataFrame: Data dengan kolom tambahan 'Skor Resiko' dan 'Kategori'
    """
    hasil_skor = []
    hasil_kategori = []
    
    for index, row in df.iterrows():
        try:
            # Ambil nilai input
            gaji = float(row['Gaji (juta)'])
            cicilan = float(row['Persentase cicilan pinjaman terhadap gaji(%)'])
            
            # PROSES FUZZY
            # Step 1: Fuzzifikasi
            mu_gaji = fuzzifikasi_gaji(gaji)
            mu_cicilan = fuzzifikasi_cicilan(cicilan)
            
            # Step 2: Inferensi
            hasil_inferensi = inferensi(mu_gaji, mu_cicilan)
            
            # Step 3: Defuzzifikasi
            skor = defuzzifikasi_centroid(hasil_inferensi)
            
            # Tentukan kategori berdasarkan skor
            if skor <= 40:
                kategori = "AMAN"
            elif skor <= 70:
                kategori = "AGAK BERESIKO"
            else:
                kategori = "BERESIKO"
            
            hasil_skor.append(round(skor, 2))
            hasil_kategori.append(kategori)
            
        except (ValueError, TypeError) as e:
            # Mengatasi data yang tidak valid
            hasil_skor.append(None)
            hasil_kategori.append("ERROR")
            print(f"  Warning: Baris {index+1} memiliki data tidak valid")
    
    # Tambahkan hasil ke dataframe
    df['Skor Resiko'] = hasil_skor
    df['Kategori'] = hasil_kategori
    
    return df

# Proses semua data
df_hasil = hitung_skor_resiko(df.copy())

In [65]:
# ============================================================
# TAMPILKAN SELURUH HASIL
# ============================================================

print("=" * 90)
print("                HASIL PENILAIAN RESIKO KREDIT SELURUH PENGAJU PINJAMAN")
print("=" * 90)
print(df_hasil.to_string(index=False))
print("=" * 90)

                HASIL PENILAIAN RESIKO KREDIT SELURUH PENGAJU PINJAMAN
 No  Gaji (juta)  Persentase cicilan pinjaman terhadap gaji(%)  Skor Resiko      Kategori
  1          3.7                                            54        81.68      BERESIKO
  2          6.2                                            48        71.08      BERESIKO
  3          2.9                                            58        82.28      BERESIKO
  4          9.5                                            44        50.00 AGAK BERESIKO
  5          4.1                                            52        82.28      BERESIKO
  6         11.0                                            37        50.00 AGAK BERESIKO
  7          5.3                                            46        80.09      BERESIKO
  8          7.8                                            41        52.94 AGAK BERESIKO
  9          3.1                                            56        81.68      BERESIKO
 10         12.4             

In [ ]:
# 10 PENGAJU DENGAN SKOR RESIKO TERKECIL

def pilih_10_terbaik(df):
    """
    Memilih 10 pengaju pinjaman dengan skor resiko kredit terkecil.
    
    Parameter:
        df (DataFrame): Data dengan skor resiko
        
    Return:
        DataFrame: 10 pengaju dengan resiko terkecil
    """
    # Filter data yang valid (bukan None/NaN)
    df_valid = df[df['Skor Resiko'].notna()].copy()
    
    # Urutkan berdasarkan skor resiko (ascending = terkecil dulu)
    df_sorted = df_valid.sort_values(by='Skor Resiko', ascending=True)
    
    # Ambil 10 teratas
    top_10 = df_sorted.head(10).reset_index(drop=True)
    
    # Tambahkan kolom peringkat
    top_10.insert(0, 'Peringkat', range(1, 11))
    
    return top_10

# Pilih 10 terbaik
df_top_10 = pilih_10_terbaik(df_hasil)

print("=" * 70)
print("         TOP 10 PENGAJU PINJAMAN DENGAN RESIKO KREDIT TERKECIL")
print("=" * 70)
print(df_top_10.to_string(index=False))
print("=" * 70)

         TOP 10 PENGAJU PINJAMAN DENGAN RESIKO KREDIT TERKECIL
 Peringkat  No  Gaji (juta)  Persentase cicilan pinjaman terhadap gaji(%)  Skor Resiko Kategori
         1  25         19.5                                            18        15.81     AMAN
         2  27         20.1                                            15        15.81     AMAN
         3  31         22.4                                             9        15.81     AMAN
         4  29         21.7                                            11        15.81     AMAN
         5  23         18.2                                            20        15.81     AMAN
         6  35         25.0                                             5        15.81     AMAN
         7  34         24.9                                             7        15.81     AMAN
         8  33         23.6                                             8        15.81     AMAN
         9  45         18.1                                            21

In [66]:
# SIMPAN OUTPUT KE FILE EXCEL

def simpan_ke_file(df_top_10, nama_file="peringkat.xlsx"):
    """
    Menyimpan hasil 10 pengaju terbaik ke file Excel.
    
    Parameter:
        df_top_10 (DataFrame): Data 10 pengaju terbaik
        nama_file (str): Nama file output
    """
    # Pilih kolom yang akan disimpan sesuai dengan ketentuan tugas
    # ID/No, Gaji, Cicilan, Skor Resiko
    kolom_output = ['Peringkat', 'No', 'Gaji (juta)', 
                    'Persentase cicilan pinjaman terhadap gaji(%)',
                    'Skor Resiko', 'Kategori']
    
    df_output = df_top_10[kolom_output].copy()
    
    # Rename kolom agar lebih rapi
    df_output.columns = ['Peringkat', 'ID/No Pengaju', 'Gaji (Juta)', 
                         'Cicilan (%)', 'Skor Resiko', 'Kategori']
    
    # Simpan ke Excel
    df_output.to_excel(nama_file, index=False)
    
    print(f" Hasil disimpan ke file: {nama_file}")
    
    return df_output

# Simpan ke file peringkat.xlsx
df_output = simpan_ke_file(df_top_10)

 Hasil disimpan ke file: peringkat.xlsx


# HASIL AKHIR: OUTPUT PROGRAM

Berikut adalah **10 pengaju pinjaman dengan skor resiko kredit terkecil** yang direkomendasikan untuk diberikan pinjaman.

In [ ]:
# TAMPILAN HASIL AKHIR (OUTPUT PROGRAM)

print()
print("╔" + "═" * 68 + "╗")
print("║" + " " * 15 + "SISTEM PENILAIAN RESIKO KREDIT" + " " * 22 + "║")
print("║" + " " * 18 + "BERBASIS FUZZY LOGIC" + " " * 29 + "║")
print("╠" + "═" * 68 + "╣")
print("║  10 PENGAJU PINJAMAN DENGAN RESIKO KREDIT TERKECIL" + " " * 16 + "║")
print("║  (Direkomendasikan untuk diberikan pinjaman)" + " " * 22 + "║")
print("╚" + "═" * 68 + "╝")
print()

print("-" * 80)
print(f"{'Peringkat':^10} | {'ID':^5} | {'Gaji (Juta)':^12} | {'Cicilan (%)':^12} | {'Skor Resiko':^12} | {'Kategori':^10}")
print("-" * 80)

for idx, row in df_output.iterrows():
    print(f"{row['Peringkat']:^10} | {row['ID/No Pengaju']:^5} | {row['Gaji (Juta)']:^12.1f} | {row['Cicilan (%)']:^12.1f} | {row['Skor Resiko']:^12.2f} | {row['Kategori']:^10}")

print("-" * 80)
print()
print("Keterangan:")
print("  • Skor Resiko: 0 (paling aman) - 100 (paling beresiko)")
print("  • Kategori AMAN        : Skor ≤ 40")
print("  • Kategori AGAK BERESIKO: 40 < Skor ≤ 70")
print("  • Kategori BERESIKO    : Skor > 70")
print()
print("File output: peringkat.xlsx")


╔════════════════════════════════════════════════════════════════════╗
║               SISTEM PENILAIAN RESIKO KREDIT                      ║
║                  BERBASIS FUZZY LOGIC                             ║
╠════════════════════════════════════════════════════════════════════╣
║  10 PENGAJU PINJAMAN DENGAN RESIKO KREDIT TERKECIL                ║
║  (Direkomendasikan untuk diberikan pinjaman)                      ║
╚════════════════════════════════════════════════════════════════════╝

--------------------------------------------------------------------------------
Peringkat  |  ID   | Gaji (Juta)  | Cicilan (%)  | Skor Resiko  |  Kategori 
--------------------------------------------------------------------------------
    1      |  25   |     19.5     |     18.0     |    15.81     |    AMAN   
    2      |  27   |     20.1     |     15.0     |    15.81     |    AMAN   
    3      |  31   |     22.4     |     9.0      |    15.81     |    AMAN   
    4      |  29   |     21.7     |

# RINGKASAN DESAIN SISTEM FUZZY

## 1. Variabel Linguistik Input
| Variabel | Linguistik | Bentuk Kurva | Batas |
|----------|------------|--------------|-------|
| Gaji (juta) | RENDAH | Trapesium | (0, 0, 4, 8) |
| Gaji (juta) | SEDANG | Segitiga | (5, 10, 15) |
| Gaji (juta) | TINGGI | Trapesium | (12, 18, 30, 30) |
| Cicilan (%) | RENDAH | Trapesium | (0, 0, 20, 35) |
| Cicilan (%) | SEDANG | Segitiga | (30, 45, 60) |
| Cicilan (%) | TINGGI | Trapesium | (50, 65, 100, 100) |

## 2. Variabel Linguistik Output
| Variabel | Linguistik | Bentuk Kurva | Batas |
|----------|------------|--------------|-------|
| Resiko Kredit | AMAN | Trapesium | (0, 0, 20, 40) |
| Resiko Kredit | AGAK BERESIKO | Segitiga | (30, 50, 70) |
| Resiko Kredit | BERESIKO | Trapesium | (60, 80, 100, 100) |

## 3. Aturan Inferensi (9 Aturan)
- **R1:** IF Gaji RENDAH AND Cicilan RENDAH THEN AGAK BERESIKO
- **R2:** IF Gaji RENDAH AND Cicilan SEDANG THEN BERESIKO
- **R3:** IF Gaji RENDAH AND Cicilan TINGGI THEN BERESIKO
- **R4:** IF Gaji SEDANG AND Cicilan RENDAH THEN AMAN
- **R5:** IF Gaji SEDANG AND Cicilan SEDANG THEN AGAK BERESIKO
- **R6:** IF Gaji SEDANG AND Cicilan TINGGI THEN BERESIKO
- **R7:** IF Gaji TINGGI AND Cicilan RENDAH THEN AMAN
- **R8:** IF Gaji TINGGI AND Cicilan SEDANG THEN AMAN
- **R9:** IF Gaji TINGGI AND Cicilan TINGGI THEN AGAK BERESIKO

## 4. Metode yang Digunakan
- **Inferensi:** Metode Mamdani (MIN-MAX)
- **Defuzzifikasi:** Metode Centroid (Center of Gravity)

## 5. Proses yang Diimplementasikan
1. Membaca data dari file (`baca_data()`)
2. Fuzzifikasi (`fuzzifikasi_gaji()`, `fuzzifikasi_cicilan()`)
3. Inferensi (`inferensi()`)
4. Defuzzifikasi (`defuzzifikasi_centroid()`)
5. Menyimpan output ke file (`simpan_ke_file()`)